In [ ]:
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Define the emotion class names
emotion_classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Set random seed for reproducibility
np.random.seed(42)

# Define data paths
train_data_dir = r'C:\Users\DEV\Desktop\Emotional Detector\test'
test_data_dir = r'C:\Users\DEV\Desktop\Emotional Detector\train'

# Define image dimensions and batch size
img_width, img_height = 48, 48
batch_size = 64

# Create data generator for training data with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create data generator for test data with rescaling only
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Load and preprocess test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    color_mode='grayscale',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 1)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Flatten())
cnn_model.add(Dense(1024, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(len(emotion_classes), activation='softmax'))

# Compile and train the CNN model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(train_generator, epochs=100, validation_data=test_generator)

# Extract features from the trained CNN model
cnn_features = cnn_model.predict(test_generator)

# Flatten the feature vectors
cnn_features = cnn_features.reshape(cnn_features.shape[0], -1)

# Create and train the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(cnn_features, test_generator.labels)

# Predict emotions for test data using the CNN + k-NN hybrid
predicted_labels = knn.predict(cnn_features)

# Print the predicted emotion and actual emotion for each image
for i in range(len(test_generator.filenames)):
    print("Image:", test_generator.filenames[i])
    print("Predicted Emotion:", emotion_classes[predicted_labels[i]])
    print("Actual Emotion:", emotion_classes[test_generator.labels[i]])
    print()

# Calculate the accuracy of the hybrid approach
accuracy = np.mean(predicted_labels == test_generator.labels)
print('Accuracy:', accuracy)